Dans ce script, nous allons tester différents modèles de type convolution Nelle pour examiner leur performance. Nous allons ensuite tester s'ils sont robustes lorsqu'on applique une rotation à l'image d'entrée.

In [1]:
import retinoto_py as fovea
args = fovea.Params(do_fovea=True)
args

Welcome on macOS-26.1-arm64-arm-64bit-Mach-O	 user laurent	Running on MPS device (Apple Silicon/MacOS)	 - macos_version = 26.1	 with device mps, pytorch==2.9.1
Random seed 1998 has been set.


Params(image_size=224, do_mask=False, do_fovea=True, rs_min=0.0, rs_max=-5.0, padding_mode='zeros', seed=1998, batch_size=64, num_workers=4, in_memory=True, model_name='resnet101', do_scratch=False, num_epochs=50, n_stop=19200, n_train_stop=19200, n_val_stop=6400, lr=0.0001, delta1=0.05, delta2=0.001, weight_decay=0.0, shuffle=True, verbose=True)

# testing each network on the validation dataset

In [2]:
%ls cached_data/32*

cached_data/32_fovea_model_name=resnet101_dataset=bbox_retrained.json
cached_data/32_fovea_model_name=resnet101_dataset=bbox_retrained.lock
cached_data/32_fovea_model_name=resnet101_dataset=bbox_retrained.pth


In [3]:
%rm cached_data/32_fovea_model_*  # FORCING RECOMPUTE
# %rm cached_data/32_fovea_model_*.lock  # FORCING RECOMPUTE
model_name = args.model_name
# for dataset in fovea.all_datasets:
for dataset in ['bbox']:# <HACK to go fat-ster ot the point
    name = f'32_fovea_model_name={model_name}_dataset={dataset}_retrained'
    model_filename, json_filename = fovea.do_learning(args, dataset, name)
    print(50*'.')

KeyboardInterrupt: 

## Model comparison

In [ ]:
results_13 = fovea.pd.read_json(args.data_cache / '13_model_comparison.json')
results_13

In [ ]:
%ls cached_data/20*

In [ ]:
model_name, dataset

In [ ]:
# results_20 = fovea.pd.read_json(args.data_cache / '20_model_name={model_name}_dataset={dataset}_retrained.json')
# results_20

In [ ]:
for dataset in fovea.all_datasets:
    name = f'20_model_name={model_name}_dataset={dataset}_retrained'
    model_filename, json_filename = fovea.do_learning(args, dataset, name)
    results = fovea.pd.read_json(json_filename)
    print(results)

## Plot learning evolution

In [ ]:
fig, ax = fovea.plt.subplots()
for dataset in fovea.all_datasets:
    color = 'r'
    lw = 1

    name = f'32_fovea_model_name={model_name}_dataset={dataset}_retrained'
    model_filename, json_filename = fovea.do_learning(args, dataset, name)

    df_train = fovea.pd.read_json(json_filename, orient='records')

    df_train_roll = df_train.rolling(window=5, min_periods=1, center=False).mean()
    ax = df_train_roll.plot(x='total_image', y='acc_train', 
                        c=color, ls='dashed', lw=lw,
                        grid=True, ax=ax, label='TRAIN: ' + args.model_name)    
    ax = df_train_roll.plot(x='total_image', y='acc_val', 
                        c=color, lw=lw,
                        grid=True, ax=ax, label='VAL: ' + args.model_name)   

In [ ]:
df_train